In [1]:
#google drive stuff
from pydrive.auth import GoogleAuth
from pydrive.drive import GoogleDrive

#for dataframe and csv creation/updating
import pandas as pd

#time info
import time

#uname info
import os
import pwd

#ip info
from urllib2 import urlopen

In [2]:
#################################
#functions

def get_username():
    return pwd.getpwuid( os.getuid() )[ 0 ]

In [3]:
#################################
#Create/load authentication files

gauth = GoogleAuth()
# Try to load saved client credentials
gauth.LoadCredentialsFile("mycreds.txt")
if gauth.credentials is None:
    # Authenticate if they're not there
    gauth.LocalWebserverAuth()
elif gauth.access_token_expired:
    # Refresh them if expired
    gauth.Refresh()
else:
    # Initialize the saved creds
    gauth.Authorize()
# Save the current credentials to a file
gauth.SaveCredentialsFile("mycreds.txt")

drive = GoogleDrive(gauth)

In [4]:
######################
#name of data file
ip_file = 'test'

In [5]:
##################################
#Check if file id exits and download.
#If file does not exist, create it.


In [6]:
if os.path.isfile('gfile_id'):
    #id file exists
    print('File exists')
    
    #get id
    check_id = open('gfile_id', 'r')
    file_id=check_id.read().decode('utf8')
    
    #init with id
    gfile = drive.CreateFile({'id': file_id})
    
    #download
    gfile.GetContentFile(ip_file+'.csv')
    
    #load df
    df = pd.read_csv(ip_file+'.csv',index_col=0)
    
else:
    print('Create file')
    ############################
    #Crate dataframe
    df = pd.DataFrame(columns=['time_stamp', 'machine','os', 'user', 'ip'])
    
    #create file
    gfile = drive.CreateFile({'title':ip_file+'.csv', 'mimeType':'text/csv'})
    gfile.Upload()
    
    #save file id for next time
    foo = gfile['id']
    f_id = open('gfile_id', 'w')
    f_id.write(foo.encode('utf8'))
    f_id.close()

File exists


In [7]:
############################
#get updated info

uname_info = os.uname()

time_stamp_temp=time.strftime("%Y-%m-%d %H:%M:%S", time.gmtime())
machine_temp = uname_info[1]
os_temp = uname_info[2]
user_temp = get_username()
ip_temp = urlopen('http://ip.42.pl/raw').read()

############################
#Create temp dataframe
df_temp=pd.DataFrame([[time_stamp_temp,machine_temp,os_temp,user_temp,ip_temp]],columns=['time_stamp', 'machine','os', 'user', 'ip'])

############################
#Append temp dataframe
df=df.append(df_temp)
df.to_csv(ip_file+'.csv',index_col=0)

In [8]:
############################
#Push file to drive

gfile.SetContentFile(ip_file+'.csv')
gfile.Upload()